In [ ]:
routes = pd.read_csv('../covid_dataset/raw/MergedRoute.csv')
dates = unique_value(routes, 'date')
patients = unique_value(routes, 'patient_id')

In [1]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 2, 4, 21, 7]
visit_types = ['karaoke', 'gas_station', 'gym', 'bakery', 'pc_cafe',
              'beauty_salon', 'school', 'church', 'bank', 'cafe',
              'bar', 'post_office', 'real_estate_agency', 'lodging',
              'public_transportation', 'restaurant', 'etc', 'store',
              'hospital', 'pharmacy', 'airport']
causes = ['community infection', 'etc', 'contact with patient', 'overseas inflow']

In [4]:
a = visit_types.index('hospitadddl')
print(a)

ValueError: 'hospitadddl' is not in list

In [39]:
EPSILON = 1e-10

def _error(actual: np.ndarray, predicted: np.ndarray):
    return actual - predicted

def _percentage_error(actual, predicted):
    return _error(actual, predicted) / (actual + EPSILON)

def mape(actual, predicted):
    return np.mean(np.abs(_percentage_error(actual, predicted)))

def mae(actual, predicted):
    return np.mean(np.abs(_error(actual, predicted)))

def mase(actual, predicted, seasonality: int = 1):
    return mae(actual, predicted) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))

def _naive_forecasting(actual, seasonality: int = 1):
    return actual[:-seasonality]

In [83]:
a = [[0, 2, 0],
    [0, 1, 0],
    [0, 3, 0]]

b = [[0, 0, 0],
    [0, 0, 0],
    [0, 0, 0]]

c = np.zeros((2, 3, 3))

c[0,:,:] = a - b

for i in range(c.shape[0]):
    for row in range(c.shape[1]):
        for col in range(c.shape[2]):
            print(c[i][row][col], end=' ')
        print()
    print()
    

In [80]:
import os.path
import csv
from csv import writer

def save_prediction_image():
    file_exists = os.path.isfile('accuracy.csv')
    print(file_exists)
    with open ('accuracy.csv', 'a') as csvfile:
        headers = ['date', 'feature', 'mape', 'rmse', 'mase']
        writer = csv.DictWriter(csvfile, delimiter=',', lineterminator='\n',fieldnames=headers)

        if not file_exists:
            print("a")
            writer.writeheader()
            
def append_list_as_row(file_name, list_of_elem):
    with open(file_name, 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(list_of_elem)

In [81]:
save_prediction_image()

# List of strings
row_contents = [32,'Shaun','Java','Tokyo','Morning']
append_list_as_row('accuracy.csv', row_contents)

True


In [97]:
from sklearn.metrics import mean_squared_error
from math import sqrt

import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
a = [[0, 0, 0],
    [0, 0, 0],
    [0, 0, 0]]

b = [[0, 55, 0],
    [0, 0, 0],
    [0, 0, 0]]
a = np.asarray(a)
b = np.asarray(b)

a[a >= 0] += 1
b[b >= 0] += 1
print(a)
print(b)




rms2 = sqrt(mean_squared_error(a, b))
mape = np.mean(np.abs((a - b) / a))
print(rms2, mape)

[[1 1 1]
 [1 1 1]
 [1 1 1]]
[[ 1 56  1]
 [ 1  1  1]
 [ 1  1  1]]
18.333333333333336 6.111111111111111


In [ ]:

# raw_data
# 여기서 places_indeices랑 patient_date_path 얻으면 됨
def get_patient_route(patient, routes):
    patient_places = routes[routes['patient_id']==patient]
    patient_dates = unique_value(patient_places, 'date')
    f_first_day = datetime.strptime(patient_dates[0], "%Y-%m-%d")
    f_last_day = datetime.strptime(patient_dates[-1], "%Y-%m-%d") + timedelta(days=3)
    f_delta = f_last_day - f_first_day
    f_duration = f_delta.days + 1

    patient_routes = []
    today = f_first_day
    while(True):
        today_str = datetime.strftime(today, "%Y-%m-%d")
        patient_day_places = patient_places[patient_places['date']==today_str]
        places_indices = combine_places(patient_day_places, counts, causes, visit_types)
        patient_routes.append([today_str, places_indices]) 

        if today == f_last_day: break
        today += timedelta(days=1)
        
    return patient_routes
        
# accumulated_data
# accumulated_patient 가져와서 작업하면 됨
def get_complete_routes(routes, dates, patients):
    f_first_day = datetime.strptime(dates[0], "%Y-%m-%d")
    f_last_day = datetime.strptime(dates[-1], "%Y-%m-%d") + timedelta(days=3)

    # 날짜별 경로 배열 생성
    today = f_first_day
    complete_routes = []
    while(True):
        today_str = datetime.strftime(today, "%Y-%m-%d")
        places = []
        complete_routes.append([today_str, places])

        if today == f_last_day: break
        today += timedelta(days=1)

    # 환자 경로 가져온 다음 날짜대로 배치
    for patient in patients:
        accumulated_routes = accumulate_patient(patient, routes)
        for each_route in accumulated_routes:
            route_day = datetime.strptime(each_route[0], "%Y-%m-%d")
            route_places = each_route[1]
            index = (route_day - f_first_day).days
            complete_routes[index][1].extend(route_places)
    return complete_routes

